# Service Titan Data Science Internship

Importing necessary libraries.

In [59]:
import pandas as pd

Rules for Conversion Table and Expiration.

In [60]:
conv_table = {
    0: 'Material',
    1: 'Equipment',
    2: 'Service',
    3: 'Other'
}
exp = [305869, 377307, 391273, 385290, 331902, 397723, 343695, 340601, 347510, 325156, 379387, 348894, 322229, 326649, 377960, 325063, 369378, 319405, 383681, 367288, 356552, 394428, 381476, 381457, 349879, 343254, 366751, 330931, 363263, 326452, 379687, 356532, 383235, 313012, 368913, 352442, 315960, 351096, 331193, 392657, 352391, 389528, 338547, 379961, 337140, 323231, 371205, 378746, 307175, 340299]

The Data Extractor Class.

In [61]:
class DataExtractor:
    def __init__(self, path):
        self.path = path
        self.df = None
        self.data = None
        
    # Reading data
    def load_data(self):
        try:
            self.data = pd.read_pickle(self.path)
            print("Data loaded")
        except FileNotFoundError:
            print("File not found")
            
    def create_dataframe(self):
        #Creating Empty DataFrame
        df = pd.DataFrame()
        #Creating Empty lists for each column of the Data Frame.
        invoice_id = []
        created_on = []
        invoiceitem_id = []
        invoiceitem_name = []
        type = []
        unit_price = []
        quantity = []
        
        #As in some places there is no value corresponding to key 'items' we need to remove them to be able to work.
        for i in self.data:
            if i.get('items'):
                pass
            else:
                self.data.remove(i)
        #Using a nested loop to fill the empty lists.
        for i in self.data:
            for j in i['items']:
                invoice_id.append(i['id']) #As we have more than 1 item corresponding to each invoice id we need to have repeating ids.
                created_on.append(i['created_on']) #The same for created one
                invoiceitem_id.append(j['item']['id'])
                invoiceitem_name.append(j['item']['name'])
                type.append(j['item']['type'])
                unit_price.append(j['item']['unit_price'])
                quantity.append(j['quantity'])
                
        #Asingn the lists to columns of the empty Data Frame.
        df['invoice_id'] = invoice_id
        df['created_on'] = created_on
        df['invoiceitem_id'] = invoiceitem_id
        df['invoiceitem_name'] = invoiceitem_name
        df['type'] = type
        df['unit_price'] = unit_price
        df['quantity'] = quantity
        
        #As all ID values are 6-digit numbers there are some IDs that have 6 digits and a letter 'O' at the end, we need to deal with it as well. After it we can convert it to type int.
        df['invoice_id'] = df['invoice_id'].astype(str) #first we need to convert it to str for str.rstrip to work
        df['invoice_id'] = df['invoice_id'].str.rstrip('O')
        df['invoice_id'] = df['invoice_id'].astype(int)
        
        #As there is a February 30th date in our created on column we cannot convert it to datetime type, that is why the most resonable solution would be simply deleting that kind of rows.
        df = df[df['created_on'] != '2019-02-30']
        df.reset_index(drop=True, inplace=True) #Resetting index to avoid further issues.
        df['created_on'] = pd.to_datetime(df['created_on'])
        
        #Doing further data type assignments.
        df['invoiceitem_name'] = df['invoiceitem_name'].astype(str)
        df['invoiceitem_id'] = df['invoiceitem_id'].astype(int)
        df['unit_price'] = df['unit_price'].astype(int)
        
        #Mapping the type column with the given conversion table.
        df['type'] = df['type'].map(conv_table)
        
        #As in the quantity column there were too many issues, for example there were negative values or string values, the most resonable solution would be deleting that rows. Here we are filling the errors with NaN and further dropping them.
        df['quantity'] = df['quantity'].apply(pd.to_numeric, errors='coerce')
        df.dropna(inplace=True)
        df.reset_index(drop=True, inplace=True)
        df['quantity'] = df['quantity'].astype(int)
        
        #Creating the total price column with the given rule.
        df['total_price'] = df['unit_price'] * df['quantity']
        df['total_price'] = df['total_price'].astype(int)
        
        #Creating the percentage in invoice column with the given rule.
        df['invoice_total'] = df.groupby('invoice_id')['total_price'].transform('sum')
        df['percentage_in_invoice'] = df['unit_price'] * df['quantity'] / df['invoice_total']
        
        #Creating the expired coulmn with the given rule.
        df['is_expired']= df['invoice_id']
        df['is_expired'] = df['is_expired'].apply(lambda x: x in exp)
        
        #Dropping Unnecessary Columns.
        df.drop(['quantity', 'invoice_total'], axis=1, inplace=True)
        self.df = df
        print('Data Frame created')
        
    def sort_data(self):
        self.df = self.df.sort_values(by=['invoice_id', 'invoiceitem_id'])
        print('Data sorted')
        
    def export_csv(self, export_path):
        self.df.to_csv(export_path, index=False)
        print('Data exported')         

In [62]:
ext = DataExtractor('invoices_new.pkl')

In [63]:
ext.load_data()

Data loaded


In [64]:
ext.create_dataframe()

Data Frame created


In [65]:
ext.sort_data()

Data sorted


In [66]:
ext.export_csv('result.csv')

Data exported
